```
bryan@ichabod mac_bucket % for file in *_dst.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/dst/
done
```


In [ ]:
df = spark.read.format("json").load("/Volumes/geodata/petra/dst/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash, replace_10e30_with_null
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    F.col("fmtest.base").alias("base"),
    F.col("fmtest.bhp").alias("bhp"),
    F.col("fmtest.bht").alias("bht"),
    F.col("fmtest.chgdate").alias("app_row_changed"),
    F.col("fmtest.choke").alias("choke"),
    F.col("fmtest.cushamt").alias("cushamt"),
    F.col("fmtest.cushtype").alias("cushtype"),
    F.col("fmtest.date").alias("date"),
    F.col("fmtest.ffp").alias("ffp"),
    F.col("fmtest.fhp").alias("fhp"),
    F.col("fmtest.fmname").alias("fmname"),
    F.col("fmtest.fsp").alias("fsp"),
    F.col("fmtest.ihp").alias("ihp"),
    F.col("fmtest.isp").alias("isp"),
    F.col("fmtest.mts").alias("mts"),
    F.col("fmtest.nummts").alias("nummts"),
    F.col("fmtest.numrecov").alias("numrecov"),
    F.col("fmtest.ohtime").alias("ohtime"),
    F.col("fmtest.recid").alias("recid"),
    F.col("fmtest.recov").alias("recov"),
    F.col("fmtest.remark").alias("remark"),
    F.col("fmtest.sitime").alias("sitime"),
    F.col("fmtest.top").alias("top"),
    F.col("fmtest.unitstype").alias("unitstype"),
    F.col("fmtest.wsn").alias("wsn"),
)


df_dst = df_flat


# enforce timestamp for dates
for col_name in ["date", "app_row_changed"]:
    df_dst = string_to_iso_date(df_dst, col_name, col_name)


# ensure real nulls
for col_name in [
    "base",
    "bhp",
    "bht",
    "choke",
    "cushamt",
    "ffp",
    "fhp",
    "fsp",
    "ihp",
    "top",
]:
    df_dst = replace_10e30_with_null(df_dst, col_name, col_name)


# add id hash
id_columns = ["repo_id", "uwi"]
df_dst = generate_hash(df_dst, "id", "dst", *id_columns)


display(df_dst)